**EECS4415N Assignment 2, Winter 2025**

**Full Name:** Nicholas Ramnaraign

**Student Number:** 211616406

**Please do not use any other libraries other than the ones imported below.**

**Please write your code in the "Your code" parts.**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive2

!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u442-b06~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u442-b06~us1-0

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from nltk.stem import PorterStemmer
import re
import math
import numpy as np


In [4]:
# Initialize Spark Session
spark = SparkSession.builder.appName("VSM Retrieval").getOrCreate()
sc = spark.sparkContext

**Read all the text files into a RDD named text_rdd**

In [75]:
# Define path to input text files (change this to your directory)
input_dir = "/content/drive/MyDrive/YourPath/CISI-IndividualDocuments/"

# Load all text files into an RDD
text_rdd = sc.wholeTextFiles(input_dir)  # Each element is (filename, content)
print("Total number of documents:")
print(text_rdd.count())

Total number of documents:
1460


In [76]:
# See the top 5 elements of text_rdd
text_rdd.take(5)


[('file:/content/drive/MyDrive/YourPath/CISI-IndividualDocuments/461.txt',
  "Information Retrieval and Processing\n    The present book embodies a change in structure and focus to reflect the fact that the reader of today's book is much more likely to be an interested college student with a great awareness of the current information revolution than was the case ten years ago.  Thus, hardware, materials, and processes used in connection with information systems are discussed first, in Chapters Two through Four.  The subject of information retrieval per se begins with Chapters Five and Six, which have to do with librarianship  and documentation.  Because of their somewhat historical slant, these chapters (along with Seven) are the only ones taken from the 1963 book which adhere to their original character.  Chapter Seven presents a simplified concept of an information system and its components, and paves the way for discussion of computerized retrieval in the chapters to follow, especia

# Part 1: Text Preprocessing

**Read a list of stopwords from a file named stop_words.lst (which is a text file with one stopword per line) from the current working directory**

In [77]:
# Read the stopwords from the stop_words.txt file and put them in a set named stop_words
# Your code

# Path to the stop_words.lst file
path_stop_words = "/content/drive/MyDrive/YourPath/stop_words.lst"

# Reads the list of stop words from the file
stop_words = set(sc.textFile(path_stop_words).collect())
# print(list(stop_words)[:10])

**Define a function to get the filename prefix without the full path name and the file extension. The prefix will be used as the document ID.**

In [78]:
def get_filename_prefix(filepath):
  """
  Extracts the filename prefix from a given filepath.
  Args:
    filepath: The full path to the file.
  Returns:
    The filename prefix (without extension or path), or None if the input is invalid.
  """
  try:
    filename = os.path.basename(filepath)  # Get the filename from the path
    filename_without_ext = os.path.splitext(filename)[0]  # Remove the extension
    return filename_without_ext
  except:
    return None


**Convert text_rdd into another RDD (doc_words_rdd) whose element is (doc_id, word_list) where doc_id is the file name prefix of a document and word_list is the list of words in the document after preprocessing. Process the content of each document in text_rdd by:**

*   keeping only the words consisting of only alphabetic letters (terms like 18, A20, 20K should be excluded)  
*   converting the words to lower cases
*   removing stopwords
*   converting the words into word stems using the Porter Stemmer in NLTK

In [79]:
# Use the Porter Stemmer from NLTK
stemmer = PorterStemmer()

# Function to preprocess text
def preprocess_text(document):
  """
  Convert a document into a list of words after preprocessing.
  Args:
    document: (the full path to the document file, the content of the file)
  Returns:
    (doc_id, a list of words after preprocessing))
  """
  filename, content = document
  words = re.findall(r"\b[a-z]+\b", content.lower())  # Tokenization

  # Your code for stopword removing and stemming

  # Initialize lists.
  filtered_words = []
  stemmed_words = []

  # Loop through words and only keep the ones that aren't stop words.
  for word in words:
    if word not in stop_words:
      filtered_words.append(word)

  # Loop through the words and apply stemming.
  for word in filtered_words:
      # stemmed_word = stemmer.stem(word)
      stemmed_words.append(stemmer.stem(word))

  # Reassign stemmed_words to filtered_words because that is what we are expected to return.
  filtered_words = stemmed_words

  # Return filtered and stemmed words.
  return (get_filename_prefix(filename), filtered_words)  # (doc_id, [word1, word2, word3, ...])

# Convert text_rdd into another RDD (doc_words_rdd) whose elment is (doc_id, word_list)
doc_words_rdd = text_rdd.map(preprocess_text)

# Part 2: Build Inverted Index

**Compute term frequency (tf_value) of each word in each document, and put the results in a RDD (tf_rdd) where each element is (word, (doc_id, tf_value)). The relative term frequency value should be used, that is, the tf_value of a word $w$ in a document $d$ should be the number of occurrences of $w$ in $d$ divided by the total number of words in $d$.**

In [80]:
# Compute Term Frequency (TF)
def compute_tf(doc_tuple):
    """
    Compute term frequency (TF) for each word in a document.
    Args:
        doc_tuple: (doc_id, [word1, word2, word3, ...])
    Returns:
        A list of (word, (doc_id, tf_value)) tuples.
    """


    # Your Code

    # Unpack the tuple to get words and doc_id.
    doc_id, words = doc_tuple

    # Caclulates the frequency of words,
    frequency_words = {}

    # Lopp through words and count the frequency, storing the value in the
    # freq_words dictionary.
    for word in words:
      if word in frequency_words:
        frequency_words[word] += 1
      else:
        frequency_words[word] = 1

    # Get the total number of words.
    num_words = len(words)

    # Initialize term frequency list.
    tf_list = []

    # Compute term frequency and add to the return list.
    # Term frequency = frequency of word / total number of words.
    # Stored as (word, (doc_id, tf)).
    for word in frequency_words:
      tf_list.append((word, (doc_id, frequency_words[word]/num_words)))

    return tf_list # York Code

# Convert doc_words_rdd into tf_rdd whiose element is (word, (doc_id, tf_value))
tf_rdd = doc_words_rdd.flatMap(compute_tf)

**From tf_rdd, compute the document frequency of each word and put the results in a RDD (df_rdd), where each element is (word, doc_count)**

In [81]:
# Compute Document Frequency (DF) per word

# Your Code


# Calcualtes the document frequency of each word
# First it convers the data into (word, doc_id) for easy processing.
# Next we remove any duplicates.
# We then convert to (word, 1) so we can sum the words.
# Then finally, we merge words and count the number merged.
df_rdd = (
    tf_rdd  # (word, (doc_id, tf_value))
    .map(lambda x: (x[0], x[1][0]))  # Extract (word, doc_id)
    .distinct()  # Remove duplicate (word, doc_id) pairs
    .map(lambda x: (x[0], 1))  # Convert to (word, 1)
    .reduceByKey(lambda x, y: x + y)  # Sum counts per word
    .sortBy(lambda x: x[0])
)

# print(df_rdd.take(5))

**From df_rdd, compute inverted document frequency (idf) of each word and put the results in another RDD idf_rdd whose element is (word, idf)**

The idf value of a word is computed as $log\frac{N}{df+1}$, where $N$ is the total number of documents and df is document frequency of the word. The "1" in denominator (df+1) prevents zero division.
Hint: use df_rdd.mapValues()

In [82]:
# Compute Inverted Document Frequency (IDF)

# Your Code

# Get the total number of documents.
num_docs = text_rdd.count()

# Calculate the inverted document frequency.
# idf = log(number of documents / (document frequency + 1))
idf_rdd = df_rdd.mapValues(lambda x: math.log(num_docs / (x + 1)))

# print(idf_rdd.take(5))

**From tf_rdd and idf_rdd, compute the tf-idf value of a word for each document and put the results in another RDD tfidf_rdd whose element is (word, (doc_id, tf-idf))**

In [83]:
# Compute TF-IDF

# Your Code

# To calculate the tf-idf we first join the tf and idf RDDs to make calculations simpler.
# We then sort for easier debugging and testing.
# Format comes out as (word, ((doc_id, tf_value), idf_value)).
tfidf_rdd = tf_rdd.join(idf_rdd).sortBy(lambda x: x[0])

# print(tfidf_rdd.take(5))


# Once merged we have access to each word, doc_id, and all the tf and idf values.
# This now makes it easy to restructure the RDD.
# We use map to rearrange the values, and caclulate the tf-idf value.
# tf-idf = tf * idf
tfidf_rdd = tfidf_rdd.map(lambda x: (x[0], (x[1][0][0], x[1][0][1] * x[1][1])))

# print(tfidf_rdd.take(5))

**Convert tfidf_rdd to an inverted index RDD whose element is (word, [(doc_id, tf-idf), (doc_id, tf-idf), ...])**

In [84]:
# Convert to Inverted Index Format

# Your Code

# Converts to inverted index format by grouping
# keys to together, and making their values a series of tuples
# formatted as (doc_id, tf-idf).
# We also sort for easier testing and debugging.
inverted_index_rdd = tfidf_rdd.groupByKey().mapValues(list).sortByKey()

print(inverted_index_rdd.take(5))

[('aacr', [('920', 0.12487346012441652), ('995', 0.17170100767107271), ('1000', 0.23380562746699263), ('1002', 0.2680210851450891), ('1003', 0.22426254063160517)]), ('abandon', [('612', 0.09364916434257922), ('848', 0.07283823893311718), ('1425', 0.046092948074863214)]), ('abbrevi', [('516', 0.07222618292330922), ('563', 0.17486339023538022), ('600', 0.048858888448120946), ('657', 0.08592425209841958), ('735', 0.048858888448120946), ('851', 0.1661202207236112), ('1326', 0.20765027590451401), ('97', 0.06644808828944448), ('442', 0.09967213243416673)]), ('aberrystwyth', [('817', 0.11367318162314546)]), ('abidjan', [('1239', 0.08241305667678046)])]


In [85]:
# Print a sample of the TF-IDF based inverted index
for word, postings in inverted_index_rdd.take(5):
    print(f"{word}: {postings}")

aacr: [('920', 0.12487346012441652), ('995', 0.17170100767107271), ('1000', 0.23380562746699263), ('1002', 0.2680210851450891), ('1003', 0.22426254063160517)]
abandon: [('612', 0.09364916434257922), ('848', 0.07283823893311718), ('1425', 0.046092948074863214)]
abbrevi: [('516', 0.07222618292330922), ('563', 0.17486339023538022), ('600', 0.048858888448120946), ('657', 0.08592425209841958), ('735', 0.048858888448120946), ('851', 0.1661202207236112), ('1326', 0.20765027590451401), ('97', 0.06644808828944448), ('442', 0.09967213243416673)]
aberrystwyth: [('817', 0.11367318162314546)]
abidjan: [('1239', 0.08241305667678046)]


In [86]:
# See how many words in the inverted index
inverted_index_rdd.count()

5545

# Part 3: Vector Space Model

**Query processing: define a function to convert a query into its tf-idf representation**

In [88]:
# Put the word IDF values in a dictionary for fast lookup during query processing
idf_lookup = idf_rdd.collectAsMap()

# Function to process a query into a sparse TF-IDF dictionary
def process_query(query):
    """
    Process a query into a sparse TF-IDF dictionary.
    Args:
        query: The query string.
    Returns:
        A dictionary mapping words to their TF-IDF values.
    """

    # York Code

    # Apply tokenization provided by professor.
    words = re.findall(r"\b[a-z]+\b", query.lower())

    # Processes a query by filtering out stop words, and stemming.
    # Initialize lists.
    filtered_words = []
    stemmed_words = []

    # Loop through each word and only keep the ones that arent stop words.
    for word in words:
      if word not in stop_words:
        filtered_words.append(word)

    # Stem the filtered words.
    for word in filtered_words:
      stemmed_words.append(stemmer.stem(word))

    # Count the frequency of each of the words, and store the values in a dictionary.
    frequency_words = {}
    for word in stemmed_words:
      if word in frequency_words:
        frequency_words[word] += 1
      else:
        frequency_words[word] = 1

    # Get the total number of filtered and stemmed words.
    len_words = len(stemmed_words)

    # Get the percentage of each word to total number of words, and store the values in a dictionary.
    frequency_words_percentage = {}
    for word in frequency_words:
      # percentage = frequency of words / total number of words
      frequency_words_percentage[word] = frequency_words[word] / len_words


    # Calculates the tfidf each word by multiplying by the idf value.
    query_tfidf = {}
    # Loop through each word, calculating the idf.
    for word in frequency_words_percentage:
      # Checks to make sure the idf exists for the word.
      if(idf_lookup.get(word, 0)):
        # percentage of the word * idf of word = tfidf
        query_tfidf[word] = frequency_words_percentage[word] * idf_lookup.get(word)


    return query_tfidf

**Precompute Document Norms:**
The L2 norm of each document is computed once to speed up document score computation. This reduces redundant calculations at query time. This computation should be done based on the inverted index.



In [89]:
# Precompute document norms based on their words' tf-idf values and put the norms in a Python dictionary with key being doc_id and value being the document norm

# Your Code

# Initialize dictionary to store document norms.
doc_norms = {}

# Loop through each word in the inverted index.
for word in inverted_index_rdd.collect():

  # We accumulate the squared tf-idf values.
  # If the doc_id is not yet in doc_norms, then we insert the sqared tf-idf value,
  # otherwise we add on the squared tf-idf value to the existing sum.
  for doc in word[1]:
    if doc[0] not in doc_norms:
      doc_norms[doc[0]] = doc[1] ** 2
    else:
      doc_norms[doc[0]] = doc_norms[doc[0]] + (doc[1] ** 2)

# Take the square root of each of the values to get the final
# document norm value.
for norm in doc_norms:
  doc_norms[norm] = math.sqrt(doc_norms[norm])


**Define a function that computes document scores given a query and rank the documents according to their scores.**

The score of a document **d** for query **q** is $\frac{\bf{q}\cdot \bf{d}}{\bf{||d||}}$. Note that we ignore ||q|| in the denominator because it is the same  for all documents and thus does not affect document ranking. The computation of the dot product should use the sparse representation of the query and the inverted index as we discussed in class.

In [90]:
# Function to compute document scores given a query using the vector space model
def rank_documents(query, top_k):
    """
    Rank documents based on a query using the vector space model.
    Args:
        query: The query string.
        top_k: The number of top documents to return.
    Returns:
        A list of (doc_id, score) tuples, sorted by score in descending order.
    """

    # Your Code

    # Process the query and get the resulting words.
    words = process_query(query)

    # Converts the inverted index to a dictionary.
    inverted_index = dict(inverted_index_rdd.collect())

    # Initialize document scores dictionary.
    doc_scores = {}

    # Loop through each word in the query.
    for word in words:

      # Get all the documents containing that word.
      tfidfs = inverted_index.get(word)

      # For each document we calculate the product of the document tf-idf and
      # the query tf-idf.
      # We sum these values for each document so we can then divide by the
      # doc norm afterwards.
      for tfidf in tfidfs:

        # If not yet in doc_scores, then we insert it into the dictionary,
        # otherwise we sum the product onto the existing value.
        if(tfidf[0] not in doc_scores):
          doc_scores[tfidf[0]] = tfidf[1] * words[word]
        else:
          doc_scores[tfidf[0]] += tfidf[1] * words[word]

    # Here we calculate the Cosine Similarity by dividing the sums from above
    # by the document norm.
    for score in doc_scores:
      doc_scores[score] = doc_scores[score] / doc_norms[score]

    # Initialize a list to append scores in tuple format (doc_id, score).
    ranked_docs = []

    # Add each score the the ranked list.
    for score in doc_scores:
      ranked_docs.append((score, doc_scores[score]))

    # Sort the documents based on the document scores.
    # Return the top_k number of documents.
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    ranked_docs = ranked_docs[:top_k]

    return ranked_docs

**Given a query, call the rank_documents function and print the top-10 documents**

In [91]:
# Example Query 1
query = "Specific advantages of computerized index systems."
print(process_query(query))
top_docs = rank_documents(query, 10)

# Print top 10 relevant documents
print("\nTop 10 Relevant Documents:")
print("\nDoc_ID: Document Score")
for doc, score in top_docs:
    print(f"{doc}: {score}")

{'specif': 0.5048035559809252, 'advantag': 0.7194624521176892, 'computer': 0.6872088225984647, 'index': 0.3489856339087912, 'system': 0.34510201673737084}

Top 10 Relevant Documents:

Doc_ID: Document Score
377: 0.3431657131018793
869: 0.30415914371393965
824: 0.23371892909299502
1144: 0.2224614789422014
856: 0.2212078128278848
1419: 0.21683483722162045
715: 0.20666101963028413
812: 0.20652041350135483
489: 0.19080310866870978
194: 0.18652349192687345


In [92]:
# Example Query 2
query = "How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?"
top_docs = rank_documents(query, 10)

# Print top 10 relevant documents
print("\nTop 10 Relevant Documents:")
print("\nDoc_ID: Document Score")
for doc, score in top_docs:
    print(f"{doc}: {score}")


Top 10 Relevant Documents:

Doc_ID: Document Score
1138: 0.2698882732022883
532: 0.26532484401971396
1155: 0.1958742695858231
562: 0.15561758487265828
309: 0.15488168234340965
790: 0.1425356353084291
1327: 0.14038899967739987
451: 0.1385800761817236
315: 0.1383328293087868
58: 0.13745453463399604
